#Modul 5 - Part 1: Structured Streaming

Inhaltsverzeichnis tbd

##5.1. Setup und Dataset laden

In [0]:
def create_streaming_structure(output_volume):
    spark.sql("CREATE VOLUME IF NOT EXISTS workspace.default.taxi_streaming_output;")

    os.makedirs(f"{output_volume}/streamingdata", exist_ok=True)
    os.makedirs(f"{output_volume}/checkpointdir", exist_ok=True)

def delete_streaming_structure(output_table_path, output_checkpoint_path):
    dbutils.fs.rm(output_table_path, recurse = True)
    dbutils.fs.rm(output_checkpoint_path, recurse = True)


In [0]:
import os

STREAMING_INPUT_FOLDER = "/Volumes/workspace/default/taxi_volume/jsonfolder"

STREAMING_OUTPUT_VOLUME = "/Volumes/workspace/default/taxi_streaming_output"

os.makedirs(f"{STREAMING_OUTPUT_VOLUME}/streamingdata", exist_ok=True)
os.makedirs(f"{STREAMING_OUTPUT_VOLUME}/checkpointdir", exist_ok=True)

#create_streaming_setup(output_volume=STREAMING_OUTPUT_VOLUME)

STREAMING_OUTPUT_FOLDER = f"{STREAMING_OUTPUT_VOLUME}/streamingdata"
STREAMING_CHECKPOINT = f"{STREAMING_OUTPUT_VOLUME}/checkpointdir"

#delete_streaming_structure(STREAMING_OUTPUT_FOLDER, STREAMING_CHECKPOINT)

## 5.2. Streaming Quelle erzeugen

Gehe zu Notebook Helper/Modul 5 - Streaming Input

##5.3. Stream einlesen

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define schema (important for streaming)
schema = StructType([
    StructField("ride_id", StringType(), True),
    StructField("taxi_id", StringType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("pickup_datetime", StringType(), True),
    StructField("dropoff_datetime", StringType(), True)
])

# Create streaming DataFrame
df_stream = spark.readStream \
    .schema(schema) \
    .json(STREAMING_INPUT_FOLDER)


In [0]:
""" table version
streaming_df = (
    spark.readStream
         .table(input_path)
)
"""

## 5.4. Stream verarbeiten durch Logik

In [0]:
from pyspark.sql.functions import avg, count

agg_df = (
    df_stream
        .groupBy("passenger_count")
        .agg(
            count("*").alias("trip_count"),
            avg("fare_amount").alias("avg_fare")
        )
)


##5.5. Stream schreiben in Zieltabelle 

In [0]:
query = (
    agg_df.writeStream
         .queryName("stream_demo")   
         .outputMode("complete")              # replace results on each trigger
         .option("checkpointLocation", STREAMING_CHECKPOINT)
         .format("delta")
         .trigger(availableNow=True)
         .start(STREAMING_OUTPUT_FOLDER)
)


In [0]:
%sql
select * from workspace.streaming_output.outputtable